## Title
Data Raw Extraction 

### By:
Santiago Puerta - Juan Gómez

### Date:
2024-05-07

### Description:

This notebook collects movie data from the TMDb API. It downloads raw data such as titles, genres, and release dates. The goal is to build a clean and updated dataset for future recommendations.

# Questions

1- ¿Cual es el objetivo del problema?
- Desarrollar un sistema de recomendación basado en contenido que sugiera películas similares a una que el usuario está viendo o considerando ver, con el fin de mejorar la experiencia de exploración en una plataforma tipo catálogo.

2- ¿Cómo se usará su solución?
- La solución se usará de forma dinámica durante la navegación del usuario. Cuando un usuario selecciona una película o entra a su ficha, el sistema recomienda títulos similares automáticamente, ayudando a descubrir contenido relevante en ese momento.
 
3- ¿Cuáles son las soluciones actuales (si las hay)?
- Soluciones similares existen en plataformas como Netflix, Prime Video o Disney+, que recomiendan películas relacionadas basadas en contenido (sinopsis, género, actores, etc.) y/o comportamiento de usuarios. El sistema propuesto simula este enfoque, pero con una fuente abierta (TMDb) y control sobre la incorporación de títulos.
 
4- ¿Cómo se debe enmarcar este problema (supervisado / no supervisado, en línea / fuera de línea, etc.)
- No supervisado, fuera de línea con actualizaciones periódicas.

5- ¿Cómo se debe medir el desempeño o el rendimiento de la solución, una primera intuicion?
- Por la relevancia de las recomendaciones, por ejemplo, usando métricas como precisión, recall, o calidad de similitud percibida.

6- ¿La medida de desempeño está alineada con el objetivo del problema?
- Sí, evaluar qué tan relevantes son las películas recomendadas se alinea con el objetivo de sugerir contenido similar.

7- ¿Cuál sería el desempeño o rendimiento mínimo necesario para alcanzar el objetivo del problema?
- Recomendaciones coherentes con los géneros, sinopsis y palabras clave de la película base, promoviendo además títulos no populares.

8- ¿Cuáles son los problemas parecidos? ¿Se puede reutilizar experiencias o herramientas ya creadas?
- Problemas similares: sistemas de recomendación de libros o música. Se pueden reutilizar herramientas como TF-IDF, similitud coseno y técnicas de filtrado basado en contenido.

9- ¿Hay experiencia del problema disponible?
- Sí, es un problema ampliamente tratado en sistemas de recomendación. Existen buenas prácticas y bibliotecas aplicables.

10- (Importante) ¿Cómo se puede resolver el problema manualmente?
- Un analista podría buscar películas similares por género, sinopsis y palabras clave de forma manual en la base de datos.

11- Hacer un listado de los supuestos que hay hasta este momento.
- La API de TMDb es confiable y actualizada.
- La similitud semántica entre películas se puede capturar con TF-IDF.
- Las películas populares ya tienen suficiente exposición.
- Las películas nuevas se deben incorporar periódicamente.
- El sistema puede operar sin retroalimentación explícita del usuario.

12- Cual es la fuente de los datos?
- API pública de TMDb (https://developer.themoviedb.org/reference/intro/getting-started). 

13- Como se actualizan los datos?
- Mediante ingestas periódicas usando peticiones a endpoints como /discover/movie, filtrando títulos no existentes en el sistema.

14- Cada cuanto tiempo se actualizan los datos
- No se especifica una frecuencia exacta, pero se menciona que es una ingesta incremental periódica.

# Data download

## 1. Imports and configuration


In [1]:
from pathlib import Path

DATA_DIR = Path.cwd().resolve().parents[1]

In [2]:
import time
from datetime import datetime

In [3]:
import pandas as pd
import requests
from dotenv import load_dotenv
from loguru import logger
from omegaconf import OmegaConf

In [4]:
load_dotenv()

True

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
extraction_config = OmegaConf.load(DATA_DIR / "conf/data_extraction/extraction.yml")
headers = extraction_config.api.headers
timeout = extraction_config.api.timeout
HTTP_OK = extraction_config.api.http_ok

## 2. Get movies from a paginated endpoint

In [7]:
def get_movies(endpoint: str, pages: int, label: str | None = None) -> list:
    """
    Get movies from a paginated TMDb endpoint.

    Args:
        endpoint: API path (e.g., "/movie/popular")
        pages: number of pages to read
        label: optional label to tag the source

    Returns:
        List of movies
    """
    logger.info(f"Fetching movies from {endpoint}...")

    movies = []
    for page in range(1, pages + 1):
        url = f"https://api.themoviedb.org/3{endpoint}?language=es-ES&page={page}"
        response = requests.get(url, headers=headers, timeout=timeout)
        if response.status_code == HTTP_OK:
            results = response.json().get("results", [])
            for movie in results:
                if label:
                    movie["source"] = label
            movies.extend(results)
        time.sleep(0.2)

    logger.info(f"Fetched {len(movies)} movies from {endpoint}.")
    return movies

## 3. Build base dataset with recent movies

In [8]:
# def build_dataset_base(pages: int = 400, days_range: int = 120) -> pd.DataFrame:
#     """
#     Build a base dataset with recent and valid movies.
#     Assign a system entry date to each movie.

#     Args:
#         pages: number of pages to read
#         days_range: number of days to spread entry dates

#     Returns:
#         DataFrame with movies and entry date
#     """
#     logger.info(f"Building dataset with {pages} pages and {days_range} days range...")

#     movies = []
#     for page in range(1, pages + 1):
#         url = (
#             f"https://api.themoviedb.org/3/discover/movie"
#             f"?sort_by=release_date.desc&vote_count.gte=10&page={page}"
#         )
#         response = requests.get(url, headers=headers, timeout=timeout)
#         if response.status_code == HTTP_OK:
#             data = response.json().get("results", [])
#             for movie in data:
#                 movie["source"] = "exploratory"
#             movies.extend(data)
#         else:
#             print(f"Error on page {page}: {response.status_code}")

#     df = pd.DataFrame(movies).drop_duplicates(subset="id").reset_index(drop=True)

#     # Assign a system entry date (spread across last N days)
#     entry_dates = [
#         pd.Timestamp.today().normalize() - timedelta(days=int(x))
#         for x in np.random.randint(0, days_range, size=len(df))
#     ]
#     df["entry_date"] = entry_dates
#     df["was_ingested"] = False

#     logger.info(f"Dataset built with {len(df)} movies.")
#     return df

## 4. Get popular movie IDs from TMDb

In [9]:
def get_popular_ids(pages: int = 10) -> set:
    """
    Get IDs of currently popular movies.

    Args:
        pages: number of pages to fetch from popular list

    Returns:
        Set of movie IDs
    """
    logger.info("Fetching popular movie IDs...")

    popular_movies = get_movies("/movie/popular", pages)

    logger.info(f"Fetched {len(popular_movies)} popular movies.")
    return set(movie["id"] for movie in popular_movies)

## 5. Create the movie dataset

In [10]:
# df_movies = build_dataset_base()

## 6. Get popular IDs and flag each movie

In [11]:
# popular_ids = get_popular_ids()
# df_movies["is_popular"] = df_movies["id"].isin(popular_ids)

## 7. Enrich with movie details

In [ ]:
def enrich_movie_details(movie_id: int) -> dict:
    """
    Get detailed info for one movie using /movie/{id}.

    Returns a dictionary with selected fields.
    """
    logger.info(f"Enriching details for movie ID {movie_id}...")

    url = f"https://api.themoviedb.org/3/movie/{movie_id}?language=es-ES"
    response = requests.get(url, headers=headers, timeout=timeout)

    if response.status_code == HTTP_OK:
        data = response.json()
        return {
            "id": movie_id,
            "runtime": data.get("runtime"),
            "budget": data.get("budget"),
            "revenue": data.get("revenue"),
            "status": data.get("status"),
            "original_language": data.get("original_language"),
            "tagline": data.get("tagline"),
            "genres": [g["name"] for g in data.get("genres", [])],
            "spoken_languages": [
                lang["name"] for lang in data.get("spoken_languages", [])
            ],
        }
    else:
        logger.error(f"Error getting details for ID {movie_id}")
        return {
            "id": movie_id,
            "runtime": None,
            "budget": None,
            "revenue": None,
            "status": None,
            "original_language": None,
            "tagline": None,
            "genres": [],
            "spoken_languages": [],
        }

## 8. Apply detail enrichment to all movies

In [13]:
# enriched_data = [enrich_movie_details(mid) for mid in df_movies["id"]]
# df_enriched = pd.DataFrame(enriched_data)
# df = df.drop(columns=["original_language"], errors="ignore")
# df = df.merge(df_enriched, on="id", how="left")

## 9. Enrich with keywords for each movie

In [14]:
def get_keywords(movie_id: int) -> list:
    """
    Get keyword list from /movie/{id}/keywords.
    """
    logger.info(f"Getting keywords for movie ID {movie_id}...")

    url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords"
    response = requests.get(url, headers=headers, timeout=timeout)

    if response.status_code == HTTP_OK:
        data = response.json()
        return [kw["name"] for kw in data.get("keywords", [])]
    else:
        logger.error(f"Error getting keywords for ID {movie_id}")
        return []

## 10. Apply keyword enrichment

In [15]:
# df_movies["keywords"] = df_movies["id"].apply(get_keywords)

In [16]:
# df_movies.to_parquet(DATA_DIR / "data/01_raw/movies_dataset.parquet", index=False)

# Periodic incremental ingestion

In [17]:
df_movies = pd.read_parquet(DATA_DIR / "data/01_raw/movies_dataset_2025-05-11.parquet")

## 11. Fetch and enrich new movies (dynamic ingestion)

In [18]:
def fetch_new_movies(pages: int = 50, existing_ids: set | None = None) -> pd.DataFrame:
    """
    Get new movies not present in the current dataset.
    Assign today's date as entry_date.
    """
    logger.info("Fetching new movies...")

    movies = []
    for page in range(1, pages + 1):
        url = (
            f"https://api.themoviedb.org/3/discover/movie"
            f"?sort_by=release_date.desc&vote_count.gte=10&page={page}"
        )
        r = requests.get(url, headers=headers, timeout=timeout)
        if r.status_code == HTTP_OK:
            for movie in r.json().get("results", []):
                if existing_ids is None or movie["id"] not in existing_ids:
                    movie["source"] = "exploratory"
                    movie["entry_date"] = pd.Timestamp.today().normalize()
                    movies.append(movie)
        time.sleep(0.2)

    logger.info(f"Fetched {len(movies)} new movies.")

    df = pd.DataFrame(movies).drop_duplicates(subset="id").reset_index(drop=True)
    df["was_ingested"] = True

    return df

## 12. Enrich new movies with details and keywords

In [19]:
def enrich_movies(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add details and keywords to new movies.
    """
    logger.info("Enriching movies...")

    details = [enrich_movie_details(mid) for mid in df["id"]]
    df = df.drop(columns=["original_language"], errors="ignore")
    df = df.merge(pd.DataFrame(details), on="id", how="left")
    df["keywords"] = df["id"].apply(get_keywords)

    logger.info(f"Enriched {len(df)} movies.")
    return df

## 13. Simulate one ingestion run

In [20]:
existing_ids = set(df_movies["id"])

In [21]:
# Fetch and enrich only new movies
df_new_movies = fetch_new_movies(pages=50, existing_ids=existing_ids)

2025-05-12 20:44:09.223 | INFO     | __main__:fetch_new_movies:6 - Fetching new movies...
2025-05-12 20:44:40.481 | INFO     | __main__:fetch_new_movies:23 - Fetched 4 new movies.


In [ ]:
if not df_new_movies.empty:
    df_new_movies["is_popular"] = df_new_movies["id"].isin(get_popular_ids())
    df_new_movies = enrich_movies(df_new_movies)
    df_movies = (
        pd.concat([df_movies, df_new_movies])
        .drop_duplicates(subset="id")
        .reset_index(drop=True)
    )

2025-05-12 20:44:40.514 | INFO     | __main__:get_popular_ids:11 - Fetching popular movie IDs...
2025-05-12 20:44:40.515 | INFO     | __main__:get_movies:13 - Fetching movies from /movie/popular...
2025-05-12 20:44:46.187 | INFO     | __main__:get_movies:27 - Fetched 200 movies from /movie/popular.
2025-05-12 20:44:46.189 | INFO     | __main__:get_popular_ids:15 - Fetched 200 popular movies.
2025-05-12 20:44:46.194 | INFO     | __main__:enrich_movies:5 - Enriching movies...
2025-05-12 20:44:46.195 | INFO     | __main__:enrich_movie_details:7 - Enriching details for movie ID 1112417...
2025-05-12 20:44:46.691 | INFO     | __main__:enrich_movie_details:7 - Enriching details for movie ID 1313194...
2025-05-12 20:44:47.289 | INFO     | __main__:enrich_movie_details:7 - Enriching details for movie ID 1371789...
2025-05-12 20:44:47.573 | INFO     | __main__:enrich_movie_details:7 - Enriching details for movie ID 1297453...
2025-05-12 20:44:48.066 | INFO     | __main__:get_keywords:5 - Gettin

In [23]:
df_movies.head(5)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,source,entry_date,was_ingested,is_popular,runtime,budget,revenue,status,tagline,genres,spoken_languages,keywords
0,False,/1ikqGTVjXA9wkDsESVVzpLP8H1r.jpg,"[28, 80, 53]",1144430,fr,Balle perdue 3,Car genius Lino returns to conclude his vendet...,219.2462,/qycPITRqXgPai7zj1gKffjCdSB5.jpg,2025-05-06,Last Bullet,False,8.100,10,exploratory,2025-01-09,False,True,112,0,0,Released,,"[Acción, Crimen, Suspense]",[Français],[]
1,False,/iznPd7PLnCBA1G50M4DuA9wvCIy.jpg,[35],1307520,es,La más fan,"Canceled in Hollywood, star Lana Cruz heads to...",98.7524,/wgUmsekYPOt9ZQ8ero91qRnmhQY.jpg,2025-05-01,The Biggest Fan,False,5.100,12,exploratory,2025-03-08,False,True,91,0,0,Released,,[Comedia],[Español],"[obssesive fan, comedy thriller, celebratory, ..."
2,False,/ioMxoDUyaRtMZPgoOU5wJkErtKS.jpg,"[18, 80]",1242686,en,Salvable,When a battered boxer past his prime finds his...,18.2723,/cAPIS05UGqsTwiu0Qjs0WlIUW1l.jpg,2025-05-01,Salvable,False,7.100,10,exploratory,2025-03-10,False,False,101,0,0,Released,,"[Drama, Crimen]",[English],"[boxing, illegal boxing, sports drama]"
3,False,/sulb7RwIiO77n1XNM2VhldUFDW1.jpg,"[80, 53, 28]",1060046,te,హిట్: ది థర్డ్ కేస్,"Arjun Sarkaar, an SP in HIT at Visakhapatnam, ...",9.9606,/wT9tGyFol4RBwkjESXUWeBdnLJn.jpg,2025-04-30,HIT: The Third Case,False,8.800,10,exploratory,2025-02-17,False,False,157,7000000,0,Released,,"[Crimen, Suspense, Acción]",[],"[investigation, violent death, brutal murder]"
4,False,/bVm6udIB6iKsRqgMdQh6HywuEBj.jpg,"[53, 28]",1233069,de,Exterritorial,"When her son vanishes inside a US consulate, e...",599.2458,/jM2uqCZNKbiyStyzXOERpMqAbdx.jpg,2025-04-29,Exterritorial,False,6.735,213,exploratory,2025-01-30,False,True,109,0,0,Released,,"[Suspense, Acción]","[Deutsch, English]","[frankfurt am main, germany, conspiracy, missi..."


In [24]:
today_str = datetime.today().strftime("%Y-%m-%d")
filename = DATA_DIR / f"data/01_raw/movies_dataset_{today_str}.parquet"
df_movies.to_parquet(filename, index=False)